In [1]:
import pandas as pd
import numpy as np
import openpyxl
import re
from tqdm import tqdm
import os

In [2]:
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import RegexpTokenizer
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.tokenize import WhitespaceTokenizer
# from nltk.tokenize import WordPunctTokenizer

from nltk.corpus import state_union
from nltk.tokenize import PunktSentenceTokenizer, word_tokenize
from nltk.tag import pos_tag

from nltk.stem import WordNetLemmatizer
from nltk import ne_chunk

In [3]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('tagsets')
nltk.download('maxent_ne_chunker')
nltk.download('words')

[nltk_data] Downloading package punkt to /Users/kimmonica/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/kimmonica/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/kimmonica/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/kimmonica/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package tagsets to
[nltk_data]     /Users/kimmonica/nltk_data...
[nltk_data]   Package tagsets is already up-to-date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /Users/kimmonica/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to /Users/kimmonica/nltk_data...
[nltk_data]   Pa

True

### def

In [4]:
def clean_text(text): 
    """ 한글, 영문, 숫자만 남기고 제거한다. 
    :param text: 
    :return: 
    """ 
    try:
        text = text.replace(".", " ").strip() 
        text = text.replace("·", " ").strip() 
        pattern = '[^ ㄱ-ㅣ가-힣|0-9|a-zA-Z]+' 
        text = re.sub(pattern=pattern, repl='', string=text)
    except:
        text = '숫자없음' 
    return text 

In [5]:
def clean_words(text): 
    """ 한글, 영문, 숫자만 남기고 제거한다. 
    :param text: 
    :return: 
    """ 
    try:
        text = text.replace(".", " ").strip() 
        text = text.replace("·", " ").strip() 
        pattern = '[^0-9]+' 
        text = re.sub(pattern=pattern, repl='', string=text) 
        text = int(text)
    except:
        text = '숫자없음'
    return text 

In [6]:
stop_words = set(stopwords.words('english')) 

#### dataset

In [18]:
df_path = '/Users/kimmonica/Documents/GitHub/Text_Mining/common_sense_movie/dataset/'
file_list = os.listdir(df_path)
file_list_py = [file for file in file_list if file.endswith('.xlsx')]

df = pd.DataFrame()
for i in file_list_py:
    data = pd.read_excel(df_path + i)
    df = pd.concat([df,data])
    
df = df.reset_index(drop = True)

In [19]:
df = df.drop(columns='Unnamed: 0')
df.head()

,movie_cnt,movie_title,age,reviews
0,245번,Andi Mack,Child,-Why do most of the kids in my school like thi...
1,245번,Andi Mack,Child,-that show is rated TV-MA
2,245번,Andi Mack,Child,"-I haven't even watched the first episodes, bu..."
3,245번,Andi Mack,Child,"-Brah, this is possibly the worst TV show I ha..."
4,245번,Andi Mack,Child,-So everyone is saying that everything about t...


In [108]:
df.to_csv('org_reviews.csv')

In [109]:
/pwd

'/Users/kimmonica/Documents/GitHub/Text_Mining/common_sense_movie'

#### data loading

In [20]:
data = df.copy()

In [21]:
data.head()

,movie_cnt,movie_title,age,reviews
0,245번,Andi Mack,Child,-Why do most of the kids in my school like thi...
1,245번,Andi Mack,Child,-that show is rated TV-MA
2,245번,Andi Mack,Child,"-I haven't even watched the first episodes, bu..."
3,245번,Andi Mack,Child,"-Brah, this is possibly the worst TV show I ha..."
4,245번,Andi Mack,Child,-So everyone is saying that everything about t...


In [22]:
df['age'].value_counts()

Child                      94647
++++Child                  57020
Teen, 13 years old         21511
++++Teen, 13 years old     12002
Teen, 14 years old         11268
Kid, 11 years old           9678
Kid, 12 years old           8483
Kid, 10 years old           7258
++++Teen, 14 years old      6312
Teen, 15 years old          6066
++++Kid, 11 years old       5614
Kid, 9 years old            4928
++++Kid, 12 years old       4527
++++Kid, 10 years old       4280
++++Teen, 15 years old      3091
++++Kid, 9 years old        2811
Kid, 8 years old            2608
Teen, 16 years old          2365
Teen, 17 years old          1605
++++Kid, 8 years old        1565
++++Teen, 16 years old      1229
Kid, 7 years old             995
++++Teen, 17 years old       691
++++Kid, 7 years old         688
Kid, 6 years old             498
++++Kid, 6 years old         377
Adult, 18 years old          185
Kid, 0 years old             170
++++Kid, 0 years old         136
++++Adult, 18 years old      122
Kid, 5 yea

In [23]:
data['age'].value_counts()

Child                      94647
++++Child                  57020
Teen, 13 years old         21511
++++Teen, 13 years old     12002
Teen, 14 years old         11268
Kid, 11 years old           9678
Kid, 12 years old           8483
Kid, 10 years old           7258
++++Teen, 14 years old      6312
Teen, 15 years old          6066
++++Kid, 11 years old       5614
Kid, 9 years old            4928
++++Kid, 12 years old       4527
++++Kid, 10 years old       4280
++++Teen, 15 years old      3091
++++Kid, 9 years old        2811
Kid, 8 years old            2608
Teen, 16 years old          2365
Teen, 17 years old          1605
++++Kid, 8 years old        1565
++++Teen, 16 years old      1229
Kid, 7 years old             995
++++Teen, 17 years old       691
++++Kid, 7 years old         688
Kid, 6 years old             498
++++Kid, 6 years old         377
Adult, 18 years old          185
Kid, 0 years old             170
++++Kid, 0 years old         136
++++Adult, 18 years old      122
Kid, 5 yea

In [24]:
data['age'] = data['age'].apply(clean_text)
data['age'] = data['age'].apply(clean_words )

In [25]:
data= data[data['age']!='숫자없음']
data['age'].value_counts()

13    33513
14    17580
11    15292
12    13010
10    11538
15     9157
9      7739
8      4173
16     3594
17     2296
7      1683
6       875
18      307
0       306
5       130
2       110
3        80
4        74
1         8
Name: age, dtype: int64

In [26]:
data['age'] = data['age'].astype(int)

In [27]:
data = data[data['age']<9]
data = data[data['age']>2]
data.age.value_counts()

8    4173
7    1683
6     875
5     130
3      80
4      74
Name: age, dtype: int64

In [28]:
data.to_csv('[230617]child_data.csv')

#### preprocessing

In [29]:
data.head()

,movie_cnt,movie_title,age,reviews
574,264번,Ant-Man,8,-I personally enjoyed this movie. It was funny...
575,264번,Ant-Man,8,"-Clever, funny, cool Marvel movie that has lot..."
576,264번,Ant-Man,8,-A vary good movie. It is a superhero movie so...
577,264번,Ant-Man,8,-Ant-man is funny and a little violent. The vi...
578,264번,Ant-Man,8,-I watched Ant Man in theatres in Switzerland ...


##### sentence dividing

In [30]:
def sent_number(text):
    try:
        num = len(sent_tokenize(text))
    except:
        num = '오류'
    return num

In [31]:
data.shape

(7015, 4)

In [32]:
data['sent_number'] = data['reviews'].apply(sent_number)
data.head()

,movie_cnt,movie_title,age,reviews,sent_number
574,264번,Ant-Man,8,-I personally enjoyed this movie. It was funny...,3
575,264번,Ant-Man,8,"-Clever, funny, cool Marvel movie that has lot...",1
576,264번,Ant-Man,8,-A vary good movie. It is a superhero movie so...,3
577,264번,Ant-Man,8,-Ant-man is funny and a little violent. The vi...,3
578,264번,Ant-Man,8,-I watched Ant Man in theatres in Switzerland ...,13


In [33]:
data['reviews'] = data['reviews'].apply(clean_text)

In [34]:
data.head()

,movie_cnt,movie_title,age,reviews,sent_number
574,264번,Ant-Man,8,I personally enjoyed this movie It was funny ...,3
575,264번,Ant-Man,8,Clever funny cool Marvel movie that has lots o...,1
576,264번,Ant-Man,8,A vary good movie It is a superhero movie so ...,3
577,264번,Ant-Man,8,Antman is funny and a little violent The viol...,3
578,264번,Ant-Man,8,I watched Ant Man in theatres in Switzerland w...,13


In [35]:
/pwd

'/Users/kimmonica/Documents/GitHub/Text_Mining/common_sense_movie'

In [36]:
data.to_csv('[230617]child_reviews_sentnum.csv')

In [37]:
full_result = []
for i in tqdm(data.reviews.to_list()):
    tokenizer = WhitespaceTokenizer()
    word_tokens = tokenizer.tokenize(i)
    
    result= []
    for word in word_tokens:
        # word = word.lower()
        if word not in stop_words:
            lemm = WordNetLemmatizer()
            word = lemm.lemmatize(word, pos = 'v')
            result.append(word)
    
    full_result.append(result)
    


100%|██████████| 7015/7015 [00:01<00:00, 5293.98it/s]


In [38]:
full_pos = []
for for_pos in tqdm(full_result):
        k = pos_tag(for_pos)
        ner_sentence = ne_chunk(k) ## NER 개체명인식
        full_pos.append(ner_sentence)

100%|██████████| 7015/7015 [00:30<00:00, 231.20it/s]


In [39]:
print(len(full_pos))

7015


In [40]:
data.shape

(7015, 5)

In [41]:
data['full_pos'] = full_pos
data.head()

,movie_cnt,movie_title,age,reviews,sent_number,full_pos
574,264번,Ant-Man,8,I personally enjoyed this movie It was funny ...,3,"[(I, PRP), (personally, RB), (enjoy, VBP), (mo..."
575,264번,Ant-Man,8,Clever funny cool Marvel movie that has lots o...,1,"[[(Clever, NNP)], (funny, JJ), (cool, NN), [(M..."
576,264번,Ant-Man,8,A vary good movie It is a superhero movie so ...,3,"[(A, DT), (vary, JJ), (good, JJ), (movie, NN),..."
577,264번,Ant-Man,8,Antman is funny and a little violent The viol...,3,"[[(Antman, NNP)], (funny, VBD), (little, JJ), ..."
578,264번,Ant-Man,8,I watched Ant Man in theatres in Switzerland w...,13,"[(I, PRP), (watch, VBP), [(Ant, NNP), (Man, NN..."


In [42]:
final_pos =[]

for i in tqdm(full_pos):
    try:
        if len(i)>=0:
            test = str(i)
            test = test.replace('\n','')
            # print(test)

            test_list= test.split('  ')
            result= []
            for i in test_list:
                
                if i[1].isupper()==True:
                    pass
                else:
                    result.append(i)
            org_result = []
            for i in result:
                text = str(i)
                text = text.split('/')[0]
                word = text.lower()
                if word not in stop_words:
                    lemm = WordNetLemmatizer()
                    word = lemm.lemmatize(word, pos = 'v')

                    # if len(word)>2:
                    org_result.append(word)
            final_pos.append(org_result)
    except:
        final_pos.append('passed')
        pass

print(len(final_pos))

100%|██████████| 7015/7015 [00:00<00:00, 11031.71it/s]

7015


In [43]:
data['final_pos'] = final_pos

In [44]:
data.to_csv('[230617]fianl_pos.csv')

In [45]:
final_pos_tag = []

for i in tqdm(final_pos):
    try:
        i = pos_tag(i)
        final_pos_tag.append(i)
    except:
        i = '오류'
        final_pos_tag.append(i)
        pass

print(len(final_pos_tag))

100%|██████████| 7015/7015 [00:05<00:00, 1334.40it/s]

7015


In [46]:
data['final_pos_tag'] = final_pos_tag

In [47]:
data.to_csv('[230617]final_pos_tag.csv')

In [50]:
/pwd

'/Users/kimmonica/Documents/GitHub/Text_Mining/common_sense_movie'

##### ne_chunk로 NER한 후, 재tagging

In [48]:
data.head()

,movie_cnt,movie_title,age,reviews,sent_number,full_pos,final_pos,final_pos_tag
574,264번,Ant-Man,8,I personally enjoyed this movie It was funny ...,3,"[(I, PRP), (personally, RB), (enjoy, VBP), (mo...","[personally, enjoy, movie, funny, plot, decent...","[(personally, RB), (enjoy, JJ), (movie, NN), (..."
575,264번,Ant-Man,8,Clever funny cool Marvel movie that has lots o...,1,"[[(Clever, NNP)], (funny, JJ), (cool, NN), [(M...","[funny, cool, movie, lot, scifi, action]","[(funny, JJ), (cool, JJ), (movie, NN), (lot, N..."
576,264번,Ant-Man,8,A vary good movie It is a superhero movie so ...,3,"[(A, DT), (vary, JJ), (good, JJ), (movie, NN),...","[vary, good, movie, superhero, movie, theres, ...","[(vary, JJ), (good, JJ), (movie, NN), (superhe..."
577,264번,Ant-Man,8,Antman is funny and a little violent The viol...,3,"[[(Antman, NNP)], (funny, VBD), (little, JJ), ...","[funny, little, violent, violence, pretty, sta...","[(funny, JJ), (little, JJ), (violent, JJ), (vi..."
578,264번,Ant-Man,8,I watched Ant Man in theatres in Switzerland w...,13,"[(I, PRP), (watch, VBP), [(Ant, NNP), (Man, NN...","[watch, theatres, brother, mom, review, mine, ...","[(watch, NN), (theatres, VBZ), (brother, JJR),..."


In [49]:
data.final_pos_tag.tolist()[0]

[('personally', 'RB'),
 ('enjoy', 'JJ'),
 ('movie', 'NN'),
 ('funny', 'JJ'),
 ('plot', 'NN'),
 ('decent', 'NN'),
 ('thankfully', 'RB'),
 ('probably', 'RB'),
 ('others', 'NNS'),
 ('wasnt', 'RB'),
 ('much', 'JJ'),
 ('term', 'NN'),
 ('sexual', 'JJ'),
 ('innuendos', 'NN'),
 ('kiss', 'JJ'),
 ('comment', 'NN'),
 ('relate', 'NN'),
 ('topic', 'NN'),
 ('overall', 'JJ'),
 ('wasnt', 'NN'),
 ('bad', 'JJ'),
 ('iron', 'NN'),
 ('man', 'NN')]

In [241]:
import nltk

# Example sentence
sentence = "Apple is looking at buying U.K. startup for $1 billion"

# Tokenize the sentence
tokens = nltk.word_tokenize(sentence)

# Perform part-of-speech tagging
pos_tags = nltk.pos_tag(tokens)

# Perform named entity recognition
chunked = nltk.ne_chunk(pos_tags)

# Remove named entities from the original sentence
filtered_sentence = ' '.join([w[0] for w in pos_tags if not hasattr(w, 'label')])

print(filtered_sentence)

Apple is looking at buying U.K. startup for $ 1 billion


### RRE

In [51]:
df = pd.read_csv('/Users/kimmonica/Documents/GitHub/Text_Mining/common_sense_movie/[230617]child_data.csv')
df.head()

,Unnamed: 0,movie_cnt,movie_title,age,reviews
0,574,264번,Ant-Man,8,-I personally enjoyed this movie. It was funny...
1,575,264번,Ant-Man,8,"-Clever, funny, cool Marvel movie that has lot..."
2,576,264번,Ant-Man,8,-A vary good movie. It is a superhero movie so...
3,577,264번,Ant-Man,8,-Ant-man is funny and a little violent. The vi...
4,578,264번,Ant-Man,8,-I watched Ant Man in theatres in Switzerland ...


In [52]:
data = df[['age', 'reviews']]
print(data.shape)
data = data.drop_duplicates()
print(data.shape)
data.to_csv('final_use.csv')

(7015, 2)
(2231, 2)
